# optionm

In [ ]:
// // test

open testing

()



In [ ]:
open rust_operators

()



## default_value

In [ ]:
inl default_value d =
    optionm.defaultWith d

()



In [ ]:
// // test

None
|> default_value 3i32
|> _equal 3i32

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{3} / expected: %A{3}"
    ()
method0()



## (/??)

In [ ]:
inl (/??) a b =
    a |> default_value b

()



In [ ]:
// // test

None /?? 3i32
|> _equal 3i32

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{3} / expected: %A{3}"
    ()
method0()



## default_with

In [ ]:
inl default_with fn =
    function Some x => x | None => fn ()

()



In [ ]:
// // test

None
|> default_with (fun () => 3i32)
|> _equal 3i32

let rec method0 () : unit =
    let v0 : string = $"__expect / actual: %A{3} / expected: %A{3}"
    ()
method0()



## choose

In [ ]:
inl choose fn a b =
    match a, b with
    | Some x, Some y => fn x y |> Some
    | _ => None

()



In [ ]:
// // test

(Some 2i32, Some 3)
||> choose (+)
|> _equal (Some 5)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec method0 () : unit =
    let v0 : US0 = US0_1(5)
    let v1 : US0 = US0_1(5)
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v1}"
    ()
method0()



## iter

In [ ]:
inl iter fn = function
    | Some x => fn x
    | None => ()

()



In [ ]:
// // test

inl n = mut 1i32
inl fn =
    fun n' =>
        n <- *n + n'
Some 1i32 |> iter fn
None |> iter fn
*n
|> _equal 2i32

type Mut0 = {mutable l0 : int32}
let rec method0 () : unit =
    let v0 : Mut0 = {l0 = 1} : Mut0
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    let v3 : int32 = v0.l0
    let v4 : string = $"__expect / actual: %A{v3} / expected: %A{2}"
    let v5 : bool = v3 = 2
    let v6 : bool = v5 = false
    if v6 then
        failwith<unit> v4
method0()



## option'

In [ ]:
nominal option' t = $"`t option"

inl unbox forall t. (x : option' t) : option t =
    inl some x : option t = Some x
    inl none : option t = None
    $"!x |> Option.map !some |> Option.defaultValue !none"

inl box forall t. (x : option t) : option' t =
    match x with
    | Some x => $"Some !x"
    | None => $"None"

()



In [ ]:
// // test

inl x = Some 3i32
inl y : option i32 = None
inl x' = x |> box |> unbox
inl y' = y |> box |> unbox
(x', y') |> _equal (x, y)

type [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : int32
let rec closure0 () (v0 : int32) : US0 =
    US0_1(v0)
and method0 () : unit =
    let v0 : int32 option = Some 3
    let v1 : (int32 -> US0) = closure0()
    let v2 : US0 = US0_0
    let v3 : US0 = v0 |> Option.map v1 |> Option.defaultValue v2
    let v4 : int32 option = None
    let v5 : US0 = US0_0
    let v6 : US0 = v4 |> Option.map v1 |> Option.defaultValue v5
    let v7 : US0 = US0_1(3)
    let v8 : US0 = US0_0
    let v9 : string = $"__expect / actual: %A{struct (v3, v6)} / expected: %A{struct (v7, v8)}"
    let v14 : bool =
        match v3 with
        | US0_1(v12) -> (* Some *)
            let v13 : bool = v12 = 3
            v13
        | _ ->
            false
    let v16 : bool =
        if v14 then
            match v6 with
            | US0_0 -> (* None *)
                true
            | _ ->
                false
        else
            false
    let v17 : bool = v16 = false
    if v17 then
        failwi

## map

In [ ]:
inl map forall t u. (fn : t -> u) (x : option' t) : option' u =
    inl x = join x
    inl fn = join fn
    !\($'"!x.map(|x| !fn(x))"')

()



## flatten

In [ ]:
inl flatten x =
    match x with
    | Some (Some x) => Some x
    | _ => None

()



## unwrap

In [ ]:
inl unwrap forall t. (x : option' t) : t =
    !\($'"!x.unwrap()"')

()



## unwrap_or

In [ ]:
inl unwrap_or forall t. (def : t) (x : option' t) : t =
    !\($'"!x.unwrap_or(!def)"')

()



## rc_upgrade

In [ ]:
inl rc_upgrade forall t. (x : rust.weak_rc t) : option (rust.rc t) =
    inl x = join x
    !\($'"std::rc::Weak::upgrade(&!x)"')
    |> unbox

()

